<a href="https://colab.research.google.com/github/deskertw/Test2025/blob/main/mybip39.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

精簡版：


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
此程式將 12 或 24 字的 BIP39 助記詞轉換成更短的單一字串，
編碼結果以 1 個字母作為前綴（代表助記詞字數與進位方式）、
中間為編碼資料、尾端加上 1 個字符的驗證碼，
最終格式為：

    前綴字母 + 編碼資料 + 驗證碼

【前綴字母定義】
  - 'A'：12 字助記詞，Base58
  - 'B'：24 字助記詞，Base58
  - 'C'：12 字助記詞，Base16
  - 'D'：24 字助記詞，Base16

※ 注意：此方法僅用於簡單混淆，不適用於高安全性用途。
"""

import sys

# 定義兩種進位表示的字元表
BASE16_ALPHABET = "0123456789abcdef"
BASE58_ALPHABET = "123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz" # Corrected Base58 alphabet (removed 0 and O)

# Embedded BIP39 wordlist (English) - Loaded from bip39_english.txt
try:
    with open('/content/bip39-english.txt', 'r') as f:
        BIP39_WORDLIST = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    print("錯誤：找不到 bip39-english.txt 檔案，請確認檔案路徑是否正確。", file=sys.stderr)
    BIP39_WORDLIST = [] # Set to empty list if file not found


def load_wordlist():
    """
    直接使用內嵌的 BIP39 單字列表。
    :return: 單字列表 (list of str)
    """
    if not BIP39_WORDLIST:
         print("錯誤：單字列表為空，無法進行編碼/解碼。", file=sys.stderr)
         return []

    if len(BIP39_WORDLIST) != 2048:
        print("警告：內嵌單字列表數量非預期（應為 2048 個），請確認列表內容。", file=sys.stderr)
    return BIP39_WORDLIST


def int_to_base(num, alphabet):
    """
    將整數轉換為指定進位表示的字串。
    :param num: 非負整數
    :param alphabet: 使用的字元表（例如 BASE58_ALPHABET 或 BASE16_ALPHABET）
    :return: 編碼後的字串
    """
    if num == 0:
        return alphabet[0]
    digits = []
    base = len(alphabet)
    while num:
        num, rem = divmod(num, base)
        digits.append(alphabet[rem])
    return ''.join(reversed(digits))

def base_to_int(s, alphabet):
    """
    將指定進位表示的字串轉換回整數。
    :param s: 編碼字串
    :param alphabet: 使用的字元表
    :return: 轉換後的整數
    """
    num = 0
    base = len(alphabet)
    for char in s:
        idx = alphabet.find(char)
        if idx == -1:
            raise ValueError(f"不合法的字元: {char}")
        num = num * base + idx
    return num

def compute_checksum(data, alphabet, length=1):
    """
    使用簡單方法計算驗證碼：
    將 data 中每個字符的 Unicode 值相加後 mod 進位字元表長度，
    得到的數值對應字元表中的字符即為驗證碼。
    :param data: 用來計算驗證碼的字串（通常為「前綴字母 + 編碼資料」）
    :param alphabet: 使用的字元表
    :param length: 驗證碼長度（此處固定為 1）
    :return: 驗證碼字串
    """
    checksum_val = sum(ord(c) for c in data) % len(alphabet)
    # 由於 mod 後結果在 [0, len(alphabet)-1]，直接取字元表中對應字符
    return alphabet[checksum_val]

def encode_mnemonic(mnemonic, wordlist, alphabet):
    """
    將 12 或 24 字助記詞轉換成指定進位表示的字串，
    並加上前綴與 1 個字符的驗證碼。
    :param mnemonic: 助記詞（單字以空格分隔）
    :param wordlist: BIP39 單字列表
    :param alphabet: 使用的字元表（BASE58_ALPHABET 或 BASE16_ALPHABET）
    :return: 編碼後的字串，格式為 "前綴字母 + 編碼資料 + 驗證碼"
    """
    words = mnemonic.split()
    if len(words) not in (12, 24):
        print("錯誤：助記詞必須為 12 或 24 個單字。", file=sys.stderr)
        return None
    total_bits = len(words) * 11  # 每個單字 11 位元
    binary_str = ""
    for word in words:
        try:
            index = wordlist.index(word)
        except ValueError:
            print(f"錯誤：單字 '{word}' 不在 BIP39 單字列表中。", file=sys.stderr)
            return None
        binary_str += format(index, '011b')
    num = int(binary_str, 2)
    encoded_data = int_to_base(num, alphabet)

    # 根據助記詞字數與進位方式決定前綴字母
    if len(words) == 12 and alphabet == BASE58_ALPHABET:
        prefix = 'A'
    elif len(words) == 24 and alphabet == BASE58_ALPHABET:
        prefix = 'B'
    elif len(words) == 12 and alphabet == BASE16_ALPHABET:
        prefix = 'C'
    elif len(words) == 24 and alphabet == BASE16_ALPHABET:
        prefix = 'D'
    else:
        print("錯誤：前綴決定失敗。", file=sys.stderr)
        return None

    # 組合初步編碼結果（前綴 + 編碼資料）
    raw_encoded = prefix + encoded_data
    # 計算 1 個字符的驗證碼
    checksum = compute_checksum(raw_encoded, alphabet, length=1)
    return raw_encoded + checksum

def decode_mnemonic(encoded_str, wordlist):
    """
    將編碼後的字串（含前綴與 1 個字符驗證碼）還原成原始助記詞，
    並驗證驗證碼是否正確。
    :param encoded_str: 編碼後的字串（格式：前綴字母 + 編碼資料 + 驗證碼）
    :param wordlist: BIP39 單字列表
    :return: 還原後的助記詞字串；若驗證碼不符則回傳錯誤訊息
    """
    if not encoded_str or len(encoded_str) < 2:
        print("錯誤：輸入字串格式不正確。", file=sys.stderr)
        return None

    checksum_length = 1  # 驗證碼長度為 1
    # 分離 data_part 與提供的驗證碼
    data_part = encoded_str[:-checksum_length]
    provided_checksum = encoded_str[-checksum_length:]

    # 根據 data_part 的第一個字元決定助記詞字數與進位方式
    prefix = data_part[0]
    if prefix == 'A':
        word_count, alphabet = 12, BASE58_ALPHABET
    elif prefix == 'B':
        word_count, alphabet = 24, BASE58_ALPHABET
    elif prefix == 'C':
        word_count, alphabet = 12, BASE16_ALPHABET
    elif prefix == 'D':
        word_count, alphabet = 24, BASE16_ALPHABET
    else:
        print("錯誤：未知的前綴字母。", file=sys.stderr)
        return None

    # 驗證 checksum 是否正確
    expected_checksum = compute_checksum(data_part, alphabet, length=1)
    if provided_checksum != expected_checksum:
        print("錯誤：驗證碼不符，資料可能有誤。", file=sys.stderr)
        return None

    # 將 data_part 的前綴字母去除，剩下的即為編碼資料
    encoded_data = data_part[1:]
    total_bits = word_count * 11
    try:
        num = base_to_int(encoded_data, alphabet)
    except ValueError as e:
        print(f"錯誤：{e}", file=sys.stderr)
        return None

    binary_str = format(num, f'0{total_bits}b')
    if len(binary_str) > total_bits:
        print("錯誤：解碼後的二進位長度超過預期。", file=sys.stderr)
        return None

    words = []
    for i in range(0, total_bits, 11):
        chunk = binary_str[i:i+11]
        index = int(chunk, 2)
        if 0 <= index < len(wordlist):
            words.append(wordlist[index])
        else:
            words.append("[未知]")
    return ' '.join(words)

def main():
    wordlist = load_wordlist()
    if not wordlist: # Check if wordlist loading failed
        return

    mode = input("請輸入模式 (encode/decode): ").strip().lower()

    if mode == "encode":
        mnemonic = input("請輸入助記詞（以空格分隔，僅限 12 或 24 字）: ").strip()
        # 在 ENCODE 模式下詢問使用者是否使用 Base16；若輸入 "base16" 則選用 Base16，否則預設使用 Base58
        base_choice = input("若要使用 Base16 請輸入 'base16'，否則預設使用 Base58：").strip().lower()
        if base_choice == "base16":
            alphabet = BASE16_ALPHABET
        else:
            alphabet = BASE58_ALPHABET
        encoded = encode_mnemonic(mnemonic, wordlist, alphabet)
        if encoded:
            print("\n編碼後的字串：")
            print(encoded)
    elif mode == "decode":
        encoded_str = input("請輸入編碼後的字串: ").strip()
        decoded = decode_mnemonic(encoded_str, wordlist)
        if decoded:
            print("\n還原後的助記詞：")
            print(decoded)
    else:
        print("未知的模式，請輸入 'encode' 或 'decode'。", file=sys.stderr)

# In Colab, you can call main() directly to run the script.
main()